import relevant libaries, data and split it into training and test data

In [31]:
import numpy as np
import autokeras as ak
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

X = np.load("data_train_Facial.npy", allow_pickle = True)
y = np.loadtxt("true_labels_Facial_train.csv", delimiter = ",")

X_train, X_test, y_train, y_test = train_test_split(X, y)

find best model and save it

In [9]:
'''reg = ak.ImageRegressor(max_trials = 100)
reg.fit(X_train, y_train)

model = reg.export_model()
try:
    model.save("model_autokeras", save_format = "tf")
except Exception:
    model.save("model_autokeras.h5")'''

'reg = ak.ImageRegressor(max_trials = 100)\nreg.fit(X_train, y_train)\n\nmodel = reg.export_model()\ntry:\n    model.save("model_autokeras", save_format = "tf")\nexcept Exception:\n    model.save("model_autokeras.h5")'

load best model

In [10]:
reg = load_model("model_autokeras", custom_objects = ak.CUSTOM_OBJECTS)

calculate accuracy on test set

In [25]:
y_predicted = reg.predict(X_test, verbose = 1).flatten()
accuracy = np.sum(y_test == np.round(y_predicted)) / y_test.shape[0]
print(f"Accuracy: {accuracy}")

157/157 [==============================] - 8s 53ms/step
Accuracy: 1.0
